In [17]:
#================================================================
#PREPARACION PIPELINA - EXCEL(Sheets) Clinica Prime
#================================================================
import pandas as pd
import numpy as np
import sys


file = '/mnt/c/DataScience/Projects_Python/Clinica_Prime/clientes_work.xlsx'
excel_file= pd.ExcelFile(file)
meses = excel_file.sheet_names

nombre_hoja = []
nombres_columnas = []
for mes in meses:
    df = excel_file.parse(mes)
    for col in df.columns:
        nombre_hoja.append(mes)
        nombres_columnas.append(str(col).lower())
    
df_nombres_columnas = pd.DataFrame({
    'hoja': nombre_hoja,
    'nombre_col': nombres_columnas
})



map_nombres_col = {}

nombres_correctos=['nombre', 'edad', 'teléfono', 'dni', 'sexo', 'nacimiento','distrito', 'tratamiento', 'notas', 'total', 'fecha','motivo de consulta', 'procedimiento', 'metodo de pago',]

#Notas a tener en cuneta
    # pp Eliminar si o si 
    # jo -> Hace Referencia a 
    # costo -> Es una columna Redundante que total se debe Eliminar 
    # fotos _> si o si se elimina usaulemente vacia 
    # `hora` y cualquier Parecido si o si se debe Eliminar
    #  tratamiento/procedimiento se convierte ahroa a `tratamiento`
    # `motivo de consulta` se dbe elimianr muy pocas Referencias a este
    #  NO se Sabe que son y que conteido pede que tengaun ocnteido relevante peor que haya habido un erro al digitar el nombre las siguiente columnas: 
            ## `hora.1 `, `columna 1`, `columna 10`, `.0 `, `975186567 `, 
 

/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell H86 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I49 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [18]:
count_column = df_nombres_columnas['nombre_col'].value_counts()

print(type(count_column))
print(count_column)
print('*' * 50)

<class 'pandas.core.series.Series'>
nombre_col
nombre                                                                                                                                                   17
edad                                                                                                                                                     17
teléfono                                                                                                                                                 17
dni                                                                                                                                                      17
notas                                                                                                                                                    17
sexo                                                                                                                                                     17
nacimiento       

In [19]:
# Se al ver el type se sabes que es un `Array` no un Dataframe

# Para convertirlo en un Dataframe Se Realiza `.reset_index()`

df_frecuencia = count_column.reset_index()
print('\n --- Dataframe --- ')
print(type(df_frecuencia))
print(df_frecuencia)
print('-' * 50)




 --- Dataframe --- 
<class 'pandas.core.frame.DataFrame'>
                                           nombre_col  count
0                                              nombre     17
1                                                edad     17
2                                            teléfono     17
3                                                 dni     17
4                                               notas     17
5                                                sexo     17
6                                          nacimiento     17
7                                            distrito     17
8                                               total     17
9                                                  pp     16
10                                        tratamiento     14
11                                              fecha     11
12                                     metodo de pago     10
13                                               hora     10
14                        

In [20]:
sospechosos = df_frecuencia['count'] < 11

df_sospechoso = df_frecuencia[sospechosos]

list_sospechosos = df_sospechoso['nombre_col'].tolist()

print('------ ANALISIS DE COLUMNAS SOSPECHOSAS A TENER EN CUENTA ----------')
print(len(df_sospechoso))
display(df_sospechoso)

------ ANALISIS DE COLUMNAS SOSPECHOSAS A TENER EN CUENTA ----------
16


,nombre_col,count
12,metodo de pago,10
13,hora,10
14,costo,10
15,hora.1,4
16,fotos,4
17,----------------------------------------------...,2
18,motivo de consulta,2
19,.0,1
20,columna 10,1
21,procedimiento,1


In [21]:
# Agiizar
# Nombre legibles no Deseados Se agregan a una lista llamada `eliminar_columna`
eliminar_columna = [ 'costo', #Redundante con el Total, no hace Referencia a Costo del Material
                     'fotos ',
                     'hora',
                     'hora.1',
                     'motivo de consulta',
                     'metodo de pago',
                     'metodo de pago.1',
                     'anuncio',
                     'deuda']

columnas_reemplazar =  ['tratamiento/procedimiento',
                       'procedimiento',
                       'notas.1']


map_columnas = {'tratamiento/procedimiento': 'tratamiento',
                       'procedimiento':'tratamiento',
                       'notas.1':'notas'}

In [22]:
# Convert columnas_reemplazar (dict) to its keys for exclusion
columnas_analizar = [
	col for col in list_sospechosos
	if col not in eliminar_columna and col not in columnas_reemplazar
]

In [23]:
df_columns_analizar = df_nombres_columnas.loc[df_nombres_columnas['nombre_col'].isin(columnas_analizar)]

df_columns_analizar

#--------------------- INFORMACION -------------------------
num_filas = df_columns_analizar.index
print('\n-------------- INFORMACION DE COLUMNAS A EVALAUR -------------\n')
print(f'\n Numero de columnas a Analizar {len(num_filas)}\n')
display(df_columns_analizar)




-------------- INFORMACION DE COLUMNAS A EVALAUR -------------


 Numero de columnas a Analizar 7



,hoja,nombre_col
0,ENERO,.0
61,JULIO,jo
83,AGOSTO,columna 10
101,SEPTIEMBRE,columna 1
146,DICIEMBRE,975186567
211,ABRIL 2025,----------------------------------------------...
229,MAYO 2025,----------------------------------------------...


In [24]:
for _, col in df_columns_analizar.iterrows():
    hoja = col['hoja']
    nombre_col = col['nombre_col']
    df = excel_file.parse(hoja)
    # Buscar la columna real (case-insensitive)
    columna_real = None
    for column in df.columns:
        if str(column).strip().lower() == nombre_col.strip().lower():
            columna_real = column
            break
    if columna_real is not None:
        print(f'\n ---------- Mes: {hoja} ------------------------')
        print(f'\n ---------- Columna a Analizar: {columna_real} -----------')
        print(df[columna_real].head())
        nulos = df[columna_real].isna().sum()
        print(f'\n Se tiene la Siguiente cantidad de Nulos: {nulos}')
        dtype = df[columna_real].dtype
        print(f'\n El Tipo de Dato es el Siguiente {dtype} \n')

        if dtype == 'datetime64[ns]':
            print('---- Realizar Cambio de Nombre a `fecha` -------')
            nuevo_nombre = 'fecha'
            df.rename(columns = {nombre_col:'fecha'}, inplace = True)
            print(f"Columna '{nombre_col}' cambiada a {nuevo_nombre} ")
        
        elif dtype == 'object': 
            nuevo_nombre = 'tratamiento'
            df.rename(columns = {nombre_col:'tratamiento'}, inplace = True)
            print(f"Columna '{nombre_col}' cambiada a {nuevo_nombre} ")
            print(df.colums)         
        
        df_final = pd.concat([df_final, df], ignore_index = True)
    else:
        print(f"\n[ADVERTENCIA] Columna '{nombre_col}' no encontrada en hoja '{hoja}'")
    


 ---------- Mes: ENERO ------------------------

 ---------- Columna a Analizar: .0 -----------
0   2024-01-01
1   2024-01-01
2   2024-01-03
3   2024-01-04
4   2024-01-10
Name: .0, dtype: datetime64[ns]

 Se tiene la Siguiente cantidad de Nulos: 0

 El Tipo de Dato es el Siguiente datetime64[ns] 

---- Realizar Cambio de Nombre a `fecha` -------
Columna '.0' cambiada a fecha 


NameError: name 'df_final' is not defined

In [ ]:
# Paso 1. Analalizar Hoja por Hoja Donde Se Evalua si se Tiene 

In [ ]:
df.columns



Index(['fecha', 'Hora', 'Nombre', 'Edad', 'Teléfono', 'DNI', 'Sexo',
       'Nacimiento', 'Distrito', 'Tratamiento', 'Notas', 'Metodo de pago',
       'Costo', 'TOTAL', 'PP', 'FOTOS '],
      dtype='object')

In [ ]:
# =================================================================================
# 🏛️ PROYECTO CLÍNICA PRIME - PROTOCOLO DE INTERROGATORIO REFINADO V14.0
# =================================================================================
import pandas as pd
import sys

# --- FASE 1: RECOLECCIÓN Y CLASIFICACIÓN PRELIMINAR ---
print("--- FASE 1: Recolección de Evidencia y Clasificación Preliminar ---")
try:
    file = '/mnt/c/DataScience/Projects_Python/Clinica_Prime/clientes_work.xlsx'
    excel_file = pd.ExcelFile(file)
    meses = excel_file.sheet_names

    # 1A: Recolección y Normalización (La táctica más eficiente)
    todas_las_columnas_unicas = {
        str(col).lower() for mes in meses for col in excel_file.parse(mes).columns
    }
    
    # 1B: Codificación de Reglas de Negocio
    SET_A_ELIMINAR = {'costo', 'fotos', 'hora', 'hora.1', 'motivo de consulta', 'metodo de pago', 'metodo de pago.1', 'anuncio', 'deuda', 'pp'}
    MAPA_DE_ALIAS = {'tratamiento/procedimiento': 'procedimiento', 'tratamiento': 'procedimiento', 'notas.1': 'notas', 'nombre': 'nombre_completo', 'teléfono': 'telefono'}

    # 1C: Triage para Aislar los Misterios (La corrección de tu lógica)
    misterios = [
        col for col in todas_las_columnas_unicas
        if col not in SET_A_ELIMINAR and col not in MAPA_DE_ALIAS.keys()
    ]
    # También excluimos las columnas que ya sabemos que son "pilares" para enfocarnos.
    nombres_correctos_set = {'edad', 'dni', 'sexo', 'nacimiento', 'distrito', 'notas', 'total', 'fecha', 'procedimiento'}
    misterios = [col for col in misterios if col not in nombres_correctos_set]
    
    print(f"✅ Recolección y Triage completados. {len(misterios)} misterios a investigar.")
    print(f"   -> MISTERIOS: {misterios}")

except Exception as e:
    print(f"❌ MISIÓN ABORTADA en FASE 1. Error: {e}")
    sys.exit()


--- FASE 1: Recolección de Evidencia y Clasificación Preliminar ---


/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell H86 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I49 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


✅ Recolección y Triage completados. 8 misterios a investigar.
   -> MISTERIOS: ['7', '.0', 'fotos ', 'columna 10', 'columna 1', 'jo', '-----------------------------------------------------------------------------------------------------------------------------------------------------', '975186567']


In [ ]:


# --- FASE 2: EL LABORATORIO FORENSE (La corrección de tu bucle) ---
print("\n\n--- FASE 2: Iniciando Interrogatorio de Misterios ---")

# 2A: Creamos un mapa de presencia para saber dónde vive cada columna
# (Esta es una versión simplificada del script V13.2)
mapa_presencia = {}
for col_unica in todas_las_columnas_unicas:
    hojas_con_col = []
    for mes in meses:
        # Leemos las columnas de la hoja una vez
        columnas_mes = [str(c).lower() for c in excel_file.parse(mes).columns]
        if col_unica in columnas_mes:
            hojas_con_col.append(mes)
    mapa_presencia[col_unica] = hojas_con_col

# 2B: El Bucle de Interrogatorio Robusto
for sospechoso in misterios:
    print(f"\n\n----------------------------------------------------------")
    print(f"     📂 Expediente del Misterio: '{sospechoso}'")
    print(f"----------------------------------------------------------")
    
    # Consultamos nuestro mapa para saber dónde buscar
    hojas_donde_aparece = mapa_presencia.get(sospechoso, [])
    print(f"  📄 PRESENCIA: Encontrado en {len(hojas_donde_aparece)} hoja(s): {hojas_donde_aparece}")
    
    # Visitamos cada hoja donde vive el sospechoso
    for mes in hojas_donde_aparece:
        df_mes = excel_file.parse(mes)
        
        # Búsqueda Robusta del nombre real de la columna (maneja mayúsculas/minúsculas)
        columna_real_en_df = None
        for col_original in df_mes.columns:
            if str(col_original).lower() == sospechoso:
                columna_real_en_df = col_original
                break
        
        if columna_real_en_df:
            # Recolección de Evidencia
            tipo_dato = df_mes[columna_real_en_df].dtype
            muestra = df_mes[columna_real_en_df].dropna().head(3).tolist()
            
            print(f"\n    🔎 Evidencia en la hoja '{mes}' (nombre original: '{columna_real_en_df}'):")
            print(f"      - Tipo de Dato (dtype): {tipo_dato}")
            if muestra:
                print(f"      - Muestra de Contenido: {muestra}")
            else:
                print(f"      - Muestra de Contenido: [Columna vacía o con nulos]")

print("\n\n--- INTERROGATORIO FINALIZADO ---")



--- FASE 2: Iniciando Interrogatorio de Misterios ---


/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell H86 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I49 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell H86 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I49 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/miguel/miniconda3/lib/pyth



----------------------------------------------------------
     📂 Expediente del Misterio: '7'
----------------------------------------------------------
  📄 PRESENCIA: Encontrado en 1 hoja(s): ['NOV 8NO)']

    🔎 Evidencia en la hoja 'NOV 8NO)' (nombre original: '7'):
      - Tipo de Dato (dtype): datetime64[ns]
      - Muestra de Contenido: [Timestamp('2024-11-04 00:00:00'), Timestamp('2024-11-04 00:00:00'), Timestamp('2024-11-04 00:00:00')]


----------------------------------------------------------
     📂 Expediente del Misterio: '.0'
----------------------------------------------------------
  📄 PRESENCIA: Encontrado en 1 hoja(s): ['ENERO']

    🔎 Evidencia en la hoja 'ENERO' (nombre original: '.0'):
      - Tipo de Dato (dtype): datetime64[ns]
      - Muestra de Contenido: [Timestamp('2024-01-01 00:00:00'), Timestamp('2024-01-01 00:00:00'), Timestamp('2024-01-03 00:00:00')]


----------------------------------------------------------
     📂 Expediente del Misterio: 'fotos '
--

/home/miguel/miniconda3/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell H86 is marked as a date but the serial value 6608697 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [ ]:
X = input("Agregue Data:")